# Emerging Technologies Tasks

My solutions to the Emerging Technologies tasks assessment.

## Task 1: Square Root of 2

>Write a Python function called `sqrt2` that calculates and prints to the screen the square root of 2 to 100 decimal places.
>Your code should not depend on any module from the standard library or otherwise. You should research the task first and include references and a description of your algorithm.